# SeisSol 3D dynamic rupture training example: the 2016 Mw 7.8 Kaikōura, New Zealand earthquake

This is a scaled-down version of the complex 3D dynamic rupture model published in [Ulrich et al., Nat. Comms., 2019](https://www.nature.com/articles/s41467-019-09125-w).

All files to reproduce the published high-resolution model are openly available at [zenodo](https://zenodo.org/record/2538024#.Yu948S8RphA).

The dynamic rupture earthquake scenario, eproduces key characteristics of the earthquake and constraints puzzling features inferred from high-quality observations including a large gap separating surface rupture traces, the possibility of significant slip on the subduction interface, the non-rupture of the Hope fault, and slow apparent rupture speed. This model shows that the observed rupture cascade is dynamically consistent with regional stress estimates and a crustal fault network geometry inferred from seismic and geodetic data. Our model proposes that the complex fault system operates at low apparent friction thanks to the combined effects of overpressurized fluids, low dynamic friction and stress concentrations induced by deep fault creep.
![](Kaikoura_Fig1.png)
*Fault network geometry prescribed for dynamic earthquake rupture modeling. Colors on fault surfaces indicate dipping angle (dip), highlighting the flattening with depth of the Jordan Thrust, Kekerengu, and Needles faults. All segments dip westwards, except for the Humps Fault Zone. The Hope, Culverden, and Leonard Mound faults, dipping, respectively, 70° toward NorthWest, 70° toward South, and 50° toward SouthEast, are displayed in yellow. These faults do not rupture in our dynamic rupture model. Also shown are the high-resolution topography and bathymetry (Mitchell et al., 2012), and S-wave speeds (Vs) on four cross-sections of the 3D subsurface structure (Eberhart-Phillips et al., 2010) incorporated in the model.*


## Mesh

To build 3D models with complicated geometries that are beyond the capabilities of gmsh we are using [SimModeler](http://www.simmetrix.com/index.php/simmodeler/overview), a software that is free for academic use. 
The necessary mesh generation steps include:
1. CAD 1: Creating a high-resolution topography and bathymetry which represent the model's free surface. Merging it with a simple 3D volume (e.g. a box);
2. CAD 2: Creating the complex fault network constrained by fault traces and varying fault dip and intersect all faults with each other and with topo-bathymetry in a manifold manner;
3. Mesh: Automatic volumetric meshing using unstructured tetrahedral elements. Keep in mind our two differing resolution criteria: the CFL criterion determines the required resolution for seismic wave propagation, and the dynamic rupture process zone/cohesive zone size, the region behind the rupture front where the fault strength drops from its static to dynamic level ([Day et al., 2005](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2005jb003813); for SeisSol: [Wollherr et al., 2018](https://academic.oup.com/gji/article/214/3/1556/5017447?login=true)).

![](Kaikoura_Fig3.png)
*Snapshot of the wavefield (absolute particle velocity in m/s) across the fault network at a rupture time of 55 s. The high-resolution model is discretized by an unstructured mesh accounting for three-dimensional (3D) subsurface structure and high-resolution topography and featuring refined resolution in the vicinity of the faults. It incorporates the nonlinear interaction between frictional on-fault failure, off-fault plasticity, and seismic wave propagation.*

We recorded two (non-narrated) demos of all meshing steps for a similar mesh using Simmodeler and a few simple scripts available in the SeisSol repository, which are available [here](https://drive.google.com/file/d/1Y3mTAoPTAyUMdXfTzW_Ap62u4EpJM0Zf/view?usp=sharing) and [here](https://drive.google.com/file/d/1ts3QZCWUeHMwB3ZEhUpV-CmNcG-4Vwj8/view?usp=sharing).

For this training, we provide a pre-prepared smaller mesh file (approx. 280,000 elements) and first visualize it. The prescribed resolution here is 1500 m element edge length everywhere on the faults, and static mesh-coarsening away from the fault system. The published high-resolution mesh, for comparison, consists of 29 million elements, decreasing the shortest element edge lengths to 300 m close to faults, and at most 1000 m close to topography and bathymetry resolving the seismic wavefield at frequencies up to 3 Hz in the source region.

In [ ]:
import vtk
import pyvista as pv

reader = vtk.vtkXdmfReader()
reader.SetFileName('NZmicro_full1500_small_box_only.xdmf')
reader.Update()
mesh = pv.wrap(reader.GetOutput())
elevation = mesh.elevation(low_point=(0, 0, -800), high_point=(0, 0, 2970))
pv.plot(elevation, cmap='terrain', background='grey', show_edges=False, jupyter_backend='static')

## Run the code

Now we run SeisSol using one of the parameter files provided in the Kaikoura training directory. We handle the 3D heterogeneous subsurface velocity data set using [ASAGI](https://seissol.readthedocs.io/en/latest/asagi.html), our open-source software for efficient reading, interpolation, and writing of parallel, adaptive geoinformation ([Rettenberger et al., 2016](http://dl.acm.org/citation.cfm?id=2938618)).

Please adjust the OMP_NUM_THREADS variable to match the number of cores on your PC. This simulation will run for some time. In case of trouble try allowing for more RAM (ideally 5-8 GB) in the settings of docker.

The training example uses linear slip-weakening friction for efficiency reasons. Our published high-resolution model features fast velocity-weakening rate-and-state friction ([Dunham et al. 2011](https://pubs.geoscienceworld.org/ssa/bssa/article/101/5/2296/326473/Earthquake-Ruptures-with-Strongly-Rate-Weakening)), and slightly different loading/strength parametrisations (cf. the two parameter files parametersRS.par versus parametersLSW.par). SeisSol with rate and state friction will run fast if compiled with a commercial compiler such as intel, as available on most clusters. The here provided SeisSol-to-go is currently rather slow since it has been compiled with the gnu compiler , thus the auto-vectorization of math functions is limited.

You may change to fast velocity-weakening rate-and-state friction by changing the parameter file name to 'parametersRS.par'.


In [ ]:
!OMP_NUM_THREADS=4 mpirun -n 1 SeisSol_Release_dhsw_4_elastic parametersLSW.par
# on Frontera with apptainer, replace with:
# !SEISSOL_COMMTHREAD=0 OMP_NUM_THREADS=28 mpirun -n 2 apptainer run {"~/my-training.sif"} SeisSol_Release_dhsw_4_elastic parametersLSW.par

## Visualization

We now visualize what we call the fault output generated by SeisSol. These are the modeled fields across the complex fault surfaces. Note that generating this output includes interpolation, since SeisSol is solving the governing PDEs in their weak form. We can visualise all processes occurring across the fault surface during dynamic rupture, e.g. slip rates, slip, shear and normal stresses, effective friction etc.

This is how the published high-resolution rupture looks like:
![](Kaikoura_Fig2.png)
*Overview of the simulated rupture propagation. Snapshots of the absolute slip rate are shown every 5 s. The figure focuses on four different portions of the fault system, following the rupture front as it propagates from South to North. Labels indicate remarkable dynamic features of the rupture.*

![](Kaikoura_Fig4.png)
*Source properties of the published dynamic rupture model and comparison to observational inferences. Final slip magnitude a) modeled here and b) inferred by Xu et al., 2018. Final rake angle c) modeled and d) inferred by Xu et al., 2018, e) Modeled rupture velocity. f) Modeled moment rate function compared with those inferred by Bai et al., 2017, from teleseismic and tsunami data, by Zhang et al., 2017, from seismic waveform inversion and from teleseismic data by the SCARDEC method, Vallee et al., 2011.*

Check out the [documentation](https://seissol.readthedocs.io/en/latest/fault-output.html#outputmask) for an explanation of the variable names and change 'SRs' to other quantities of interest.

In [ ]:
import vtk
import pyvista as pv
from ipywidgets import interact

reader = vtk.vtkXdmfReader()
reader.SetFileName('output/NZ_LSW-fault.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 30.0, 0.5), var=variables)
def plot(t=0.0, var='SRs'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('grey')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.show(jupyter_backend='static')

The fault output, as well as all other surface and volume outputs, can be written [asynchronously](https://seissol.readthedocs.io/en/latest/environment-variables.html#asynchronous-output), i.e. specific threads or nodes can be dedicated to write these outputs.

Next, we visualize SeisSol's so called [free surface output](https://seissol.readthedocs.io/en/latest/free-surface-output.html). Fields *v1, v2, v3* are the particle velocities and *u1, u2, u3* are the displacements in *x, y, z* direction, respectively, evaluated everywhere at the free surface. 

The writer outputs additionally a quantity called “locationFlag”, which has the values 0 and 1 when at the elastic or acoustic side of an elastic-acoustic interface (as used in [Krenz et al., 21](https://arxiv.org/abs/2107.06640)). In this manner, we can distinguish between both sides of the interface even though they have the same coordinates. In our case it has the value 2 for an ordinary free surface boundary condition.

In [ ]:
reader = vtk.vtkXdmfReader()
reader.SetFileName('output/NZ_LSW-surface.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 30.0, 0.25), var=variables)
def plot(t=0.0, var='u3'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('grey')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.show(jupyter_backend='static')

### Effect of initial loading on cascading rupture dynamics
**Task:** Change the stress shape ratio (s2ratio) from 0.15 to 0.27 and  and run the updated setup to observe the much delayed dynamic triggering of the second fault. Rename the output folder to compare 3D earthquake dynamics of both setups by opening both fault output files in ParaView. Compare also the wavefield at the free surface in both simulations.